# MODELS TO BE ENSEMBLE

In [1]:
# #Setup Gdrive file download extention 
!pip install gdown open-clip-torch faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.3 MB/s eta 0:00:00


In [2]:
!gdown --id 1BFAJfzzeaUGsPoYELS86HIutJ43D-vat

/opt/conda/lib/python3.10/site-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Access denied with the following error:

 	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1BFAJfzzeaUGsPoYELS86HIutJ43D-vat 



In [3]:
path1 = '/kaggle/working/model1.pt'

In [4]:
import os
import math

import numpy as np
 
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import open_clip

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import torchvision.transforms as T

from tqdm import tqdm
import pandas as pd
import cv2
from PIL import Image

# from vpr_utils import get_similiarity_l2
from vpr_utils import convert_indices_to_labels
from vpr_utils import map_per_set

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
device = 'cuda' if th.cuda.is_available() else 'cpu';

# Reading Dataset

In [6]:
import numpy as np
import torch as th
from PIL import Image
import copy
import numpy as np
import open_clip
import torchvision.transforms as T
import torch

def average_query_expansion(query_vecs, reference_vecs, top_k=3):
    """
    Average Query Expansion (AQE)
    Ondrej Chum, et al. "Total Recall: Automatic Query Expansion with a Generative Feature Model for Object Retrieval,"
    International Conference of Computer Vision. 2007.
    https://www.robots.ox.ac.uk/~vgg/publications/papers/chum07b.pdf
    """
    # Query augmentation
    sim_mat = torch.cdist(query_vecs, reference_vecs)
    indices = torch.argsort(sim_mat, dim=1)

    top_k_ref_mean = torch.mean(reference_vecs[indices[:, :top_k], :], dim=1)
    query_vecs = torch.cat([query_vecs, top_k_ref_mean], dim=1)

    # Reference augmentation
    sim_mat = torch.cdist(reference_vecs, reference_vecs)
    indices = torch.argsort(sim_mat, dim=1)

    top_k_ref_mean = torch.mean(reference_vecs[indices[:, 1 : top_k + 1], :], dim=1)
    reference_vecs = torch.cat([reference_vecs, top_k_ref_mean], dim=1)

    return query_vecs, reference_vecs


def db_augmentation(query_vecs, reference_vecs, top_k=3):
    """
    Database-side feature augmentation (DBA)
    Albert Gordo, et al. "End-to-end Learning of Deep Visual Representations for Image Retrieval,"
    International Journal of Computer Vision. 2017.
    https://link.springer.com/article/10.1007/s11263-017-1016-8
    """
    weights = torch.logspace(0, -2.0, top_k + 1).cuda()

    # Query augmentation
    sim_mat = torch.cdist(query_vecs, reference_vecs)

    indices = torch.argsort(sim_mat, dim=1)

    top_k_ref = reference_vecs[indices[:, :top_k], :]
    query_vecs = torch.tensordot(
        weights,
        torch.cat([torch.unsqueeze(query_vecs, 1), top_k_ref], dim=1),
        dims=([0], [1]),
    )

    # Reference augmentation
    sim_mat = torch.cdist(reference_vecs, reference_vecs)
    indices = torch.argsort(sim_mat, dim=1)

    top_k_ref = reference_vecs[indices[:, : top_k + 1], :]
    reference_vecs = torch.tensordot(weights, top_k_ref, dims=([0], [1]))
    # reference_vecs = torch.tensordot(weights, torch.cat([torch.unsqueeze(query_vecs, 1), top_k_ref], dim=1), dims=([0], [1]))

    return query_vecs, reference_vecs



def calculate_l2_distances(query, gallery):
    return np.linalg.norm(gallery - query, axis=1)

def get_k_nearest_neighbors(distances, k):
    indices = np.argsort(distances)[:k]
    return indices

In [7]:
def read_image(image_file):
    img = cv2.imread(
        image_file, cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION
    )
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if img is None:
        raise ValueError('Failed to read {}'.format(image_file))
    return img


class SubmissionDataset(Dataset):
    def __init__(self, root, annotation_file, transforms, with_bbox=False):
        self.root = root
        self.imlist = pd.read_csv(annotation_file)
        self.transforms = transforms
        self.with_bbox = with_bbox

    def __getitem__(self, index):
        cv2.setNumThreads(6)

        full_imname = os.path.join(self.root, self.imlist['img_path'][index])
        img = read_image(full_imname)

        if self.with_bbox:
            x, y, w, h = self.imlist.loc[index, 'bbox_x':'bbox_h']
            img = img[y:y+h, x:x+w, :]

        img = Image.fromarray(img)
        img = self.transforms(img)
        product_id = self.imlist['product_id'][index]
        return img, product_id

    def __len__(self):
        return len(self.imlist)


In [8]:
import numpy as np
import cv2

imgs_path = "/kaggle/input/products-10k/products-10k/development_test_data/gallery/"

files = [imgs_path + x for x in os.listdir(imgs_path) if "jpg" in x]

# List of image paths
image_paths = files

# Calculate mean and std
mean = np.zeros(3)
std = np.zeros(3)
num_images = len(image_paths)

for image_path in image_paths:
    img = cv2.imread(image_path) / 255.0  # Read and normalize image
    mean += img.mean(axis=(0, 1))
    std += img.std(axis=(0, 1))

mean /= num_images
std /= num_images

print("Mean:", mean)
print("Std:", std)


Mean: [0.75410071 0.76879095 0.7930227 ]
Std: [0.25881864 0.25062647 0.23544936]


In [9]:
def get_transform():  
    transform = T.Compose([
            T.Resize(
                size=(224, 224), 
                interpolation=T.InterpolationMode.BICUBIC,
                antialias=True),
            T.ToTensor(), 
            T.Normalize(
                mean=(0.48145466, 0.4578275, 0.40821073), 
                std=(0.26862954, 0.26130258, 0.27577711)
            )
        ])
    return transform

@th.no_grad()
def get_feature_vector(model, dataloader, epoch=10, use_cuda=True):
    features = []
    product_id = []
    
    for imgs, p_id in tqdm(dataloader):
        if use_cuda:
            imgs = imgs.cuda()
        features.append(th.squeeze(model(imgs.half())).detach().cpu().numpy().astype(np.float32))
        product_id.append(th.squeeze(p_id).detach().cpu().numpy())


    return np.concatenate(features, axis=0), np.concatenate(product_id)

# Getting the CLIP model's embedding

In [10]:
backbone = open_clip.create_model_and_transforms('ViT-H-14', None)[0].visual
backbone.load_state_dict(th.load('/kaggle/input/final-model-weights-version-01/all-model-weights/vit_h_14_0.607_224_ce.pt'))
backbone.half()   # Apply half precision to the backbone model
backbone.eval()   # Dropping unecessary layers
model1 = backbone
model1.cuda();

In [11]:
backbone = open_clip.create_model_and_transforms('convnext_xxlarge', None)[0].visual
backbone.load_state_dict(th.load('/kaggle/input/final-model-weights-version-01/all-model-weights/convnext_xxlarge_0.60_320_ce.pt'))
backbone.half()   # Apply half precision to the backbone model
backbone.eval();   # Dropping unecessary layers
model2 = backbone
model2.cuda();

In [12]:
transform = get_transform()

img_dir = "/kaggle/input/products-10k/products-10k/development_test_data"

dataset_train = SubmissionDataset(img_dir, os.path.join(img_dir, "gallery.csv"), transform)
dataloader_train = DataLoader(dataset_train, batch_size=256, num_workers=4)

dataset_test = SubmissionDataset(img_dir, os.path.join(img_dir, "queries.csv"), transform, with_bbox=True)
dataloader_test = DataLoader(dataset_test, batch_size=256, num_workers=4)

In [13]:
import numpy as np
import torch as th
from PIL import Image
import copy
import numpy as np
import open_clip
import torchvision.transforms as T
import torch

def average_query_expansion(query_vecs, reference_vecs, top_k=3):
    """
    Average Query Expansion (AQE)
    Ondrej Chum, et al. "Total Recall: Automatic Query Expansion with a Generative Feature Model for Object Retrieval,"
    International Conference of Computer Vision. 2007.
    https://www.robots.ox.ac.uk/~vgg/publications/papers/chum07b.pdf
    """
    # Query augmentation
    sim_mat = torch.cdist(query_vecs, reference_vecs)
    indices = torch.argsort(sim_mat, dim=1)

    top_k_ref_mean = torch.mean(reference_vecs[indices[:, :top_k], :], dim=1)
    query_vecs = torch.cat([query_vecs, top_k_ref_mean], dim=1)

    # Reference augmentation
    sim_mat = torch.cdist(reference_vecs, reference_vecs)
    indices = torch.argsort(sim_mat, dim=1)

    top_k_ref_mean = torch.mean(reference_vecs[indices[:, 1 : top_k + 1], :], dim=1)
    reference_vecs = torch.cat([reference_vecs, top_k_ref_mean], dim=1)

    return query_vecs, reference_vecs


def db_augmentation(query_vecs, reference_vecs, top_k=3):
    """
    Database-side feature augmentation (DBA)
    Albert Gordo, et al. "End-to-end Learning of Deep Visual Representations for Image Retrieval,"
    International Journal of Computer Vision. 2017.
    https://link.springer.com/article/10.1007/s11263-017-1016-8
    """
    weights = torch.logspace(0, -2.0, top_k + 1).cuda()

    # Query augmentation
    sim_mat = torch.cdist(query_vecs, reference_vecs)

    indices = torch.argsort(sim_mat, dim=1)

    top_k_ref = reference_vecs[indices[:, :top_k], :]
    query_vecs = torch.tensordot(
        weights,
        torch.cat([torch.unsqueeze(query_vecs, 1), top_k_ref], dim=1),
        dims=([0], [1]),
    )

    # Reference augmentation
    sim_mat = torch.cdist(reference_vecs, reference_vecs)
    indices = torch.argsort(sim_mat, dim=1)

    top_k_ref = reference_vecs[indices[:, : top_k + 1], :]
    reference_vecs = torch.tensordot(weights, top_k_ref, dims=([0], [1]))
    # reference_vecs = torch.tensordot(weights, torch.cat([torch.unsqueeze(query_vecs, 1), top_k_ref], dim=1), dims=([0], [1]))

    return query_vecs, reference_vecs



def calculate_l2_distances(query, gallery):
    return np.linalg.norm(gallery - query, axis=1)

def get_k_nearest_neighbors(distances, k):
    indices = np.argsort(distances)[:k]
    return indices

def get_distances(gallery_embeddings, query_embeddings):
    print('Processing indices...')
    
    query_embeddings = torch.from_numpy(query_embeddings).to('cuda')
    gallery_embeddings = torch.from_numpy(gallery_embeddings).to('cuda')

    concat = torch.cat((query_embeddings, gallery_embeddings), dim=0)
    center = torch.mean(concat, dim=0)
    query_embeddings = query_embeddings - center
    gallery_embeddings = gallery_embeddings - center
    gallery_embeddings = torch.nn.functional.normalize(
        gallery_embeddings, p=2.0, dim=1
    )
    query_embeddings = torch.nn.functional.normalize(query_embeddings, p=2.0, dim=1)

#     query_embeddings, gallery_embeddings = db_augmentation(
#         query_embeddings, gallery_embeddings, top_k=5
#     )
    
#     query_embeddings, gallery_embeddings = average_query_expansion(query_embeddings, gallery_embeddings, top_k=25)


    distances = torch.cdist(query_embeddings, gallery_embeddings)
    
    sorted_distances, sorted_indices = torch.sort(distances, dim=1)

    class_ranks = sorted_indices
    
    # take indexes of the most similar embeddings from the gallery_embeddings
    first_gallery_idx = class_ranks[:, 0]
    
    # take distance value of the most similar embeddings from the gallery_embeddings
    first_gallery_dstx = sorted_distances[:, 0]

    # take the most similar embedding from the gallery_embeddings by index
    rerank_embeddings1 = gallery_embeddings.index_select(0, first_gallery_idx)

    # if distance between most similar gallery and query < 0.3 
    # then add it to the new embeddings list for ranking (filter_rerank_embeddings1) 
    # else add embedding from query_embeddings
    mask1 = first_gallery_dstx < 0.3
    filter_rerank_embeddings1 = torch.where(
        mask1.view(-1, 1), rerank_embeddings1, query_embeddings
    )
    
    # averaging and ranking
    filter_rerank_embeddings = (
        0.5 * filter_rerank_embeddings1 + 0.5 * query_embeddings
    )
    distances = torch.cdist(filter_rerank_embeddings, gallery_embeddings)
    
    # then the same thing, but with the two most similar embeddings from gallery_embeddings

    sorted_distances, sorted_indices = torch.sort(distances, dim=1)
    first_gallery_idx = class_ranks[:, 0]
    first_gallery_dstx = sorted_distances[:, 0]
    second_gallery_idx = class_ranks[:, 1]
    second_gallery_dstx = sorted_distances[:, 1]

    rerank_embeddings1 = gallery_embeddings.index_select(0, first_gallery_idx)
    rerank_embeddings2 = gallery_embeddings.index_select(0, second_gallery_idx)

    mask1 = first_gallery_dstx < 0.3
    mask2 = second_gallery_dstx < 0.3

    filter_rerank_embeddings1 = torch.where(
        mask1.view(-1, 1), rerank_embeddings1, query_embeddings
    )
    filter_rerank_embeddings2 = torch.where(
        mask2.view(-1, 1), rerank_embeddings2, query_embeddings
    )

    filter_rerank_embeddings = (
        0.3 * filter_rerank_embeddings1
        + 0.3 * filter_rerank_embeddings2
        + 0.4 * query_embeddings
    )

    distances = torch.cdist(filter_rerank_embeddings, gallery_embeddings)
    
    return distances

    
def get_similiarity_l2(distances, k):

    sorted_distances = torch.argsort(distances, dim=1)
    sorted_distances = sorted_distances.cpu().numpy()[:, :1000]
    class_ranks = sorted_distances
    
    return class_ranks

In [14]:
def predict(model1, model2):
    global feature_vectors_gallery
    
    feature_vectors_gallery, labels_gallery = get_feature_vector(model1, dataloader_train, 1)
    feature_vectors_query, labels_query = get_feature_vector(model1, dataloader_test, 1)

    distances1 = get_distances(feature_vectors_gallery, feature_vectors_query)

    feature_vectors_gallery, labels_gallery = get_feature_vector(model2, dataloader_train, 1)
    feature_vectors_query, labels_query = get_feature_vector(model2, dataloader_test, 1)

    distances2 = get_distances(feature_vectors_gallery, feature_vectors_query)

    distances = torch.mul(0.7*distances1, distances2)

    indices = get_similiarity_l2(distances, 1000)

    indices = indices.tolist()
    labels_gallery = labels_gallery.tolist()
    labels_query = labels_query.tolist()

    preds = convert_indices_to_labels(indices, labels_gallery)
    score = map_per_set(labels_query, preds)
    return [indices, score]

In [15]:
[preds, score] = predict(model1, model2)
print(f'Average mAP score is : {score}')

preds = pd.DataFrame(preds)
preds.to_csv('preds.csv', index=False)

100%|██████████| 8/8 [00:58<00:00,  7.28s/it]


Processing indices...


100%|██████████| 8/8 [00:55<00:00,  6.94s/it]


Processing indices...
Average mAP score is : 0.6244616709732989
